In [184]:
import numpy as np
matrix = np.array([
    [6.20, 1.10, 0.94, 1.21],
    [1.10, 4.10, 1.30, 0.16],
    [0.94, 1.30, 7.40, 1.10],
    [1.21, 0.16, 1.10, 9.10]
])

In [185]:
matrix = np.array([
    [2.2, 1, 0.5, 2],
    [1, 1.3, 2, 1],
    [0.5, 2, 0.5, 1.6],
    [2, 1, 1.6, 2]
])

In [186]:
def Danilevsky(matrix):
    n = len(matrix)
    Ps = np.zeros((n, n, n))
    Ps[-1] = matrix.copy()
    Ms = np.array(list(np.identity(n)for i in range(n-1)))
    Ms_1 = np.array(list(np.identity(n)for i in range(n-1)))
    M = None
    for k in range(n-2, -1, -1):
    # print(Ps[k+1][k+1])
    # print(Ps[k+1][k+1][k])
        Ms[k][k] = -Ps[k+1][k+1] / Ps[k+1][k+1][k]
        Ms[k][k][k] = -Ms[k][k][k] / Ps[k+1][k+1][k]

        Ms_1[k][k] = Ps[k+1][k+1]

        Ps[k] = Ms_1[k] @ Ps[k+1] @ Ms[k]
        if M is None:
            M = Ms[k]
        else:
            M @= Ms[k]

    # print(Ms[k])
    # print(Ms_1[k])
    # print(Ps[k])

    return Ps[0][0], M

coefs, M = Danilevsky(matrix)
coefs, M

(array([  6.    ,   0.2   , -12.735 ,   2.7616]),
 array([[-0.23112481,  1.07858243,  1.65100154, -1.1587057 ],
        [ 0.08124387, -0.13671383, -1.64095812, -0.27390951],
        [ 0.23812859, -1.2627819 , -0.4131531 ,  0.36957557],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [196]:
def F(X, coefs):
    n = len(coefs)
    degrees = np.arange(n-1, -1, -1)
    # print(X, n, coefs, degrees)
    # [-2, 1, 5, -2, 7]
    # f = -2*x**4 + x**3 + 5*x**2 - 2*x + 7
    res = np.zeros(len(X))
    for x_index in range(len(X)):
        for degree_index in range(len(degrees)):
            res[x_index] += coefs[degree_index] * X[x_index]**degrees[degree_index]
    return res

def find_intervals(coefs, interval_of_finding=[-10, 10], dots_count=30):
    dots_count = int(dots_count)
    intervals = []
    n = len(coefs)
    polys = [coefs, np.polyder(coefs)]
    for i in range(1, n-1):
        polys.append(-np.polydiv(polys[i-1], polys[i])[1])

    # print(polys)

    # X = np.array([-2, -1.7, -1.4, -1, 0.8, 1.1, 1.4, 1.7, 2.0, 2.1])
    X = np.linspace(*interval_of_finding, dots_count)
    matrix = np.zeros((len(polys), len(X)))
    for poly_index in range(len(polys)):
        # print(polys[poly_index])
        # matrix[poly_index] = F(X, polys[poly_index])
        matrix[poly_index] = np.polyval(polys[poly_index], X)

    # print(matrix.T)
    # print(np.sum(matrix.T > 0, axis=1))
    # print(np.sign(matrix.T))
    # print(np.diff(np.sign(matrix.T), axis=1) != 0)
    sign_change = np.sum(np.diff(np.sign(matrix.T), axis=1) != 0, axis=1)
    difference = np.diff(sign_change, axis=0) != 0
    for diff_index in range(len(difference)):
        if difference[diff_index]:
            intervals.append([X[diff_index], X[diff_index+1]])
    return intervals
    # print(difference)
coef = np.array([1, *(-coefs)])
# print(coef)
intervals = find_intervals(coef, interval_of_finding=[-10, 10], dots_count=1e3)
intervals
# find_intervals(np.array([-2, 1, 5, -2, 7]), interval_of_finding=[-10, 10], dots_count=1e3)

[[np.float64(-1.431431431431431), np.float64(-1.411411411411411)],
 [np.float64(0.21021021021021014), np.float64(0.23023023023023015)],
 [np.float64(1.531531531531531), np.float64(1.551551551551551)],
 [np.float64(5.635635635635635), np.float64(5.655655655655655)]]

In [200]:
def Newton(a, b, coefs, epsilon=1e-6):
    xk = a
    xk_1 = b

    count = 0

    while abs(xk - xk_1) >= epsilon or abs(np.polyval(coefs, xk)) >= epsilon:
        xk_1 = xk
        xk = xk - np.polyval(coefs, xk)/np.polyval(np.polyder(coefs), xk)
        xk = np.clip(xk, a, b)
        count += 1

    return xk
main_vectors = np.zeros((len(intervals)))
for interval_index in range(len(intervals)):
    main_vectors[interval_index] = Newton(*intervals[interval_index], coef)
main_vectors

array([-1.42008659,  0.22263593,  1.54541834,  5.65203233])

In [202]:
main_vector = np.atleast_2d(main_vectors).T
main_vector

array([[-1.42008659],
       [ 0.22263593],
       [ 1.54541834],
       [ 5.65203233]])

In [188]:
coefs

array([  6.    ,   0.2   , -12.735 ,   2.7616])

In [ ]:
def Gauss_Method(A, b):
    x = np.zeros_like(b)
    for col in range(1, A.shape[1]):
        for row in range(col, A.shape[0]):
            zeroiator = (- A[row, col-1]/A[col-1, col-1])
            A[row] = A[row] + A[col-1] * zeroiator
            b[row] = b[row] + b[col-1] * zeroiator
        print(A)
        print(b)
    # print(A)
    # print(b)
    # x[-1] = b[-1] / A[-1, -1]
    # x[-2] = (b[-2] - A[-2, -1] * x[-1]) / A[-2, -2]
    # x[-3] = (b[-3] - A[-3, -1] * x[-1] - A[-3, -2] * x[-2]) / A[-3, -3]
    for col in range(A.shape[1]-1, -1, -1):
        result = 0
        for row in range(A.shape[0]-1, col, -1):
            result += A[col, row] * x[row]
        x[col] = (b[col] - result)/A[col, col]


    return x